What needs to be done

1. Load data from sendgrid users
2. Load data from csv file -- look at using pandas for this.
3. For ID in sendgrid data find the csv entry for that.

In [3]:
import csv
# Load data from sendgrid users
import json
import subprocess


user_json = "../data/sendgrid/user.json" # This is created from calling the sendgrid api. TODO: record command to create content

user = []
with open(user_json) as f:

    data = json.loads(f.read())
    user = [u['username'] for u in data]
print(user)

['rhoam-dsb-stage', 'rhoam-dsb-prod', '1shrbgabq64idgce7njqlh0dr92c87ph', 'mk-0419', 'mk-0503', 'mk-1006', 'mk-stage-0622', '1t988blfp3thl6ldds443smck0che9m8', '1udgcu2m67h87mo65vi0psctheu4ussn', '1s4idabok0jf7216dgdhn3hb30tbd5mp', '1uocc77b8uaogc7bt0o2phalshmhg8v3', '1uon0k0joc2fs5aq055n7b4i5f31ebra', '1v529ivvikohbpg8pgfihegcdjhudjng', '209jmk90v1h8igpp1vt5285qgfpcr284', '20b0vlp359m52mctag9fi1vai69q229e', '20k928mf3q718aulb2i3m13gt53pev9a', '20kcl3kaiab3kesmd6eu2gi2sffkl7v8', '20l3pgb3qr7rj1h23md3i8rdpq8nrf6k', '20m8i1m13br06eiaj1q4m853k3rtdctf', '20m9ncsbs9eapisn3m69lf92fss24dmd', '20meuq8v1fo2u9pdv93hl01b11bo6sgj', '20mfb96ir026u97i5kborl3st3hp6uok', '20mjv3gnb4enqgm2am5hll73l0gb8qsk', '20mkbop3mq2o4s2onbv94ucsu3rflhk8', '20mstu829lus4onceuh4442b4ql4c6vu', '20mtie94kgiaip8o14ghbvlcu1v4f8in', '20mu7bsvnah62nvtmgl9g6tgd9tof92q', '20mva4e635ig2l0d1rqcgkgqja1qqbak', '20mt0hm65red26qu66tu47139vkddift', '20mvjt1fukdc0js8gd6i2rmi7aef5qh6', '20n4edo4kddoed7cjvdpc3gcdodngku7', '20n90d7u4bn

In [4]:
from typing import Optional
from pydantic import BaseModel
import csv

class Job(BaseModel):
    id: Optional[str]
    cluster_id: Optional[str]
    cluster_uuid: Optional[str]
    created_at: Optional[str]
    updated_at: Optional[str]
    deleted_at: Optional[str]
    attempts: Optional[str]
    attempt_after: Optional[str]
    remove: Optional[str]
    supported_service: Optional[str]
    completed_at: Optional[str]

jobs_csv = "../data/sendgrid/jobs.csv" # This is a data bump form the sendgrid service.

jobs = []
with open(jobs_csv) as f:
    csv_data = csv.DictReader(f, delimiter="|")
    for row in csv_data:
        job = Job(**row)

        jobs.append(job)
print(jobs)


[Job(id=' 2KMW82OJSPDhdUYJkuYTnZbFsSM ', cluster_id=' 218pgnuveov6ptgn6lin3h418dnjnoca     ', cluster_uuid=' 5eee765d-1619-4cb6-979a-ef736671e686 ', created_at=' 2023-01-15 12:12:48.956677+00 ', updated_at=' 2023-01-15 12:12:55.647344+00 ', deleted_at='                               ', attempts='        0 ', attempt_after=' 2023-01-15 12:12:48.956654+00 ', remove=' f      ', supported_service=' Red Hat OpenShift Data Foundation Managed Service Provider (QE)  ', completed_at=' 2023-01-15 12:12:55.647309+00'), Job(id=' 1hsKvyg2Cr0dAGyRd1hdfpKbJqV ', cluster_id=' 1frtg733qsa7s980egn7eb97bjgv2dol     ', cluster_uuid=' 6e79a494-abba-41cd-b4dc-7476a5fff19f ', created_at=' 2020-09-22 17:09:53.641306+00 ', updated_at=' 2020-09-22 17:59:24.431922+00 ', deleted_at=' 2020-09-22 17:59:32.089926+00 ', attempts='        0 ', attempt_after=' 2020-09-22 17:09:53.641279+00 ', remove=' t      ', supported_service=' RHMI Operator                                                    ', completed_at=' 2020-0

In [5]:
simple_jobs = []
for job in jobs:
    if job.cluster_id.strip() in user:
        simple_jobs.append(job)

print(len(simple_jobs))

164


In [6]:
# get a list of all cluster that do not exist anymore

import subprocess

not_found = []
unknown = []
no_error = []

for job in simple_jobs:
    result = subprocess.run(["ocm", "get", "cluster", job.cluster_id.strip()], capture_output=True)
    if result.stderr != b'':
        err = json.loads(result.stderr.decode())
        if err["kind"] == "Error" and err["id"] == "404":
            not_found.append(job)
        else:
            unknown.append(job)
    else:
        no_error.append(job)

print(len(not_found))
print(len(unknown))
print(len(no_error))

154
0
10


In [7]:
from pprint import pprint
from collections import Counter

t = []
for job in not_found:
    t.append(job.supported_service.strip())

not_found_services = Counter(t)

t = []
for job in no_error:
    t.append(job.supported_service.strip())
no_error_services = Counter(t)

pprint(not_found_services)
pprint(no_error_services)

Counter({'Red Hat Openshift API Management': 42,
         'Red Hat OpenShift Data Foundation Managed Service Provider (QE)': 40,
         'Red Hat OpenShift Data Science': 24,
         'Red Hat OpenShift Data Foundation Managed Service Consumer (QE)': 17,
         'Managed Kafka kas-fleetshard-operator (qe)': 16,
         'Red Hat OpenShift Data Foundation Managed Service Provider': 10,
         'Red Hat OpenShift Data Foundation Managed Service Consumer': 3,
         'Red Hat OpenShift Data Foundation Managed Service Provider (dev)': 2})
Counter({'Red Hat OpenShift Data Science': 6,
         'Red Hat OpenShift Data Foundation Managed Service Provider (QE)': 2,
         'Red Hat Openshift API Management': 1,
         'Red Hat Openshift API Management (internal)': 1})


In [8]:
rhoams = []

for cluster in not_found:
    if cluster.supported_service.strip() == "Red Hat Openshift API Management":
        rhoams.append(cluster)

for cluster in rhoams:
    print(cluster.cluster_id.strip())

20nu4vlqeeqghbtp0umv5iei4hi1pq2p
20nnskmnctn4ae2ebo5u3hdd19haf2fd
20njtcci1rpsl5o2p70i0avpnpq5gaao
212r9ku4e4l4hcu7fn14ee3g3nselld7
20nec5o849i5ptcd7kfpuo5s2v8qdr3r
20kcl3kaiab3kesmd6eu2gi2sffkl7v8
2148idjpocfvavgr1a65r0i5qg2ur5sj
20npfsslu7p2bksj8qh4fdoplt29iad6
219j2d2g6li1st6769833u2f5o9gqhfb
214oo5536r0c1aild41poh2p9a9bc2tn
219nkff1rcgsi96mv64b7kacgm8kaa3n
219pe5lqrcvnai23rsgerl6urec33cec
210qlp1m2idesugju1hj3tvntjn1jg0u
219t0iq24krgkljj9l0u2r6i32nh7h1l
20oekaeur9n07o88ofr8kj814o47vj90
20oj6d80880k817hfh8dpp91s3c5is53
20ol00uiid1qe72enspo7rn3esvdlgv5
211ff0jfsjp1pcs75unh605kaob763v6
20on5qp21o7kn1qtbv9kbof5nuesu76h
211qgelicb8r3r6j045qncd409jaqer5
211s9k59c8obm68u7vojv9bg8apod3nk
2159b50ora5g94friu1vs8nbsurkaa7e
2153vfed7q0iunauaeipuq3cb0fo4jur
2155ord9k9028e92chtofi9t3il99odi
212432j3ld97bail0rpd9pg9rhcv0glt
20pot4m9fejqqok36n32miaer9g9ja2k
215eulor5h55sgj3vvdaud84mqn19u6k
20ni636cch0v4c2m94d9mhjj010n3e5i
215paotgskj72t1ecaphtcj1m8cheen0
21aomk6khl7srq91jd9tsgvednt9ejbd
20r6lto04i

In [9]:
logs = []
look_into = []

for cluster in not_found:
    resp = subprocess.run(["ocm", "get", f"/api/service_logs/v1/clusters/{cluster.cluster_uuid.strip()}/cluster_logs"], capture_output=True)
    if resp.stdout != b'':
        stdout = json.loads(resp.stdout.decode())
        if len(stdout['items']) > 0:
            logs.append((cluster, stdout['items']))
    else:
        look_into.append(cluster)

print("The results are back")
if look_into:
    print("you need to look at the look into")

if logs:
    print("Some logs were found")
else:
    print("No logs found")



The results are back
Some logs were found


In [10]:
import jsbeautifier
opts = jsbeautifier.default_options()
opts.indent_size = 2

for log in logs:
    with open(f"/tmp/logs/{log[0].cluster_id.strip()}.json", "w") as f:
        f.write(jsbeautifier.beautify(json.dumps(log[1]), opts))
        print(f"file: {log[0].cluster_id.strip()}.json ")


file: 20nm1tirfe1nr5vacoca8rjcepo0hdg1.json 
file: 20nk4b1sqelpmeailqtnai7mp6tpvtp8.json 
file: 20njs0v2vmc1h7rugns5q593stmit8ti.json 
file: 20nu4vlqeeqghbtp0umv5iei4hi1pq2p.json 
file: 20nm2nicv6o12pp9b7fevrh1j3mun659.json 
file: 20nnskmnctn4ae2ebo5u3hdd19haf2fd.json 
file: 20njtcci1rpsl5o2p70i0avpnpq5gaao.json 
file: 212k5hq8hk2vf80t4p219jdlffj9cgm4.json 
file: 20mva4e635ig2l0d1rqcgkgqja1qqbak.json 
file: 20o6cbs2t4a7bmh86vbl5o78utt28rm0.json 
file: 20o7tl8cdhkdsnhsk42lcgjauudmm4rk.json 
file: 212r9ku4e4l4hcu7fn14ee3g3nselld7.json 
file: 20njuc011vk0a6s1b3ps9i6sqm52ja48.json 
file: 20mstu829lus4onceuh4442b4ql4c6vu.json 
file: 20nec5o849i5ptcd7kfpuo5s2v8qdr3r.json 
file: 20kcl3kaiab3kesmd6eu2gi2sffkl7v8.json 
file: 20njr0covlcd281bfqh6di9bflpo3tqh.json 
file: 20o6cfd1rtvd7c612ao930pkkum48ehq.json 
file: 2148idjpocfvavgr1a65r0i5qg2ur5sj.json 
file: 20o6cen525u7ce81qv315cbh90uhv4i1.json 
file: 20vt13bbfmh56pv0gndhrfelibdrr33h.json 
file: 2199kqd937hgo5drm17edcu3kmptlk96.json 
file: 20np

In [11]:
values = []
for cluster in not_found:
    if cluster.deleted_at.strip() == "":
        values.append("Not Deleted")
    else:
        values.append("Deleted")

state = Counter(values)
pprint(state)

values = []
for cluster in not_found:
    if cluster.remove.strip() == "t":
        values.append("Removed")
    else:
        values.append("Not Removed")

state = Counter(values)
pprint(state)

Counter({'Not Deleted': 154})
Counter({'Not Removed': 154})


In [12]:
len(not_found)

154

In [13]:
for cluster in not_found:
    print(cluster.cluster_uuid)

 e3532b46-d59d-4eb2-91d3-dd07abd95284 
 2edfc6b9-ae15-47e6-b433-00417fe21d7d 
 962338ef-bf0c-47fa-8994-531b00ca5127 
 eb458928-529d-4cfb-bf8f-43cd55a3fbe9 
 21e13a05-ebc7-40c8-8112-19243ec3e41e 
 0be6d814-4581-4744-950d-ec2c9dd770d9 
 118748b8-5ead-45b3-9ec7-7b8bce914bf5 
 1b9bcac3-0aef-44d7-84b6-bfd26cf808be 
 519d2808-21fb-49a2-93d3-08239c9dd4fa 
 31da2b75-316f-40c3-bcf1-d1e3d202569d 
 ece35fe1-d992-461d-a669-8fde6c12c025 
 98d7b5a6-6049-4e63-8f34-2ad385ca4636 
 9a710955-09f7-4706-b0c0-7291eab7b081 
 eca96c1a-15d6-42d7-90d8-638589b07dc5 
 43cc4d62-72ab-46ec-8c38-32506d71e060 
 c8bf61fe-1eae-435e-8589-8fb9b44d82b7 
 258b9309-66af-450a-aff9-1c2d878a6f80 
 695735df-999c-42c5-a68c-beff52c3adfc 
 40646f8a-ed7d-4e46-ad54-79d5fafde1fa 
 47a0bb80-2b3d-4573-8e7a-4f5209faea76 
 b7b0da7a-6da8-4bdd-9c19-4116fd2a477f 
 2c65f9d0-dcd2-482d-bcf1-6f410a20e6c2 
 bddeda98-12b0-4776-a320-3b63bc1932aa 
 f5ec7353-1e5d-4fb8-a645-ef969988caa0 
 f6c0dfa9-5278-4c6e-b365-e6de0069ea51 
 17644871-04e1-4779-96f1-

In [14]:
dates = []

for cluster in not_found:
    dates.append(cluster.created_at.strip())

dates = sorted(dates)
pprint(dates)

['2022-11-02 18:39:26.303823+00',
 '2022-12-15 10:53:45.687584+00',
 '2022-12-16 14:01:51.976491+00',
 '2022-12-16 18:49:37.454941+00',
 '2022-12-18 09:30:49.309491+00',
 '2022-12-18 10:55:10.964735+00',
 '2022-12-18 16:49:49.336004+00',
 '2022-12-18 17:36:04.571136+00',
 '2022-12-18 22:46:00.498356+00',
 '2022-12-18 23:04:19.131323+00',
 '2022-12-19 08:47:19.172417+00',
 '2022-12-19 09:36:03.351827+00',
 '2022-12-19 10:14:18.147877+00',
 '2022-12-19 11:41:19.259532+00',
 '2022-12-19 12:30:34.805647+00',
 '2022-12-19 12:30:51.731463+00',
 '2022-12-19 17:35:58.447042+00',
 '2022-12-19 22:47:27.833488+00',
 '2022-12-20 01:02:37.202731+00',
 '2022-12-20 04:43:08.239116+00',
 '2022-12-20 06:48:52.49624+00',
 '2022-12-20 06:52:06.579516+00',
 '2022-12-20 07:11:40.242274+00',
 '2022-12-20 09:23:46.030154+00',
 '2022-12-20 09:35:08.426735+00',
 '2022-12-20 09:35:49.165442+00',
 '2022-12-20 10:51:49.025935+00',
 '2022-12-20 10:53:46.958393+00',
 '2022-12-20 11:16:44.102972+00',
 '2022-12-20 12

In [15]:
dl = []
for d in dates:
    dl.append(d.split(" ")[0])

cdl = Counter(dl)
pprint(cdl)

Counter({'2022-12-21': 18,
         '2022-12-20': 16,
         '2023-01-13': 12,
         '2022-12-22': 11,
         '2023-01-09': 10,
         '2022-12-19': 8,
         '2023-01-18': 7,
         '2022-12-18': 6,
         '2023-01-03': 6,
         '2023-01-12': 6,
         '2022-12-23': 5,
         '2023-01-04': 5,
         '2023-01-05': 5,
         '2023-01-10': 5,
         '2023-01-16': 5,
         '2022-12-25': 4,
         '2022-12-29': 4,
         '2023-01-17': 4,
         '2023-01-19': 3,
         '2022-12-16': 2,
         '2022-12-26': 2,
         '2023-01-06': 2,
         '2023-01-11': 2,
         '2022-11-02': 1,
         '2022-12-15': 1,
         '2022-12-28': 1,
         '2023-01-01': 1,
         '2023-01-02': 1,
         '2023-01-08': 1})


In [16]:
for cluster in not_found:
    if cluster.created_at.strip().startswith("2022-12-21"):
        print(cluster.supported_service.strip())

Red Hat Openshift API Management
Red Hat Openshift API Management
Red Hat OpenShift Data Foundation Managed Service Provider (QE)
Red Hat OpenShift Data Science
Red Hat OpenShift Data Foundation Managed Service Provider (QE)
Red Hat OpenShift Data Foundation Managed Service Provider (QE)
Red Hat Openshift API Management
Managed Kafka kas-fleetshard-operator (qe)
Red Hat OpenShift Data Science
Red Hat OpenShift Data Science
Red Hat OpenShift Data Science
Red Hat OpenShift Data Foundation Managed Service Provider (QE)
Red Hat OpenShift Data Foundation Managed Service Provider (QE)
Red Hat OpenShift Data Foundation Managed Service Consumer (QE)
Red Hat OpenShift Data Foundation Managed Service Consumer (QE)
Red Hat OpenShift Data Foundation Managed Service Consumer (QE)
Red Hat Openshift API Management
Red Hat Openshift API Management


In [17]:
failed_pipelines = ["20oj6d80880k817hfh8dpp91s3c5is53", "20ol00uiid1qe72enspo7rn3esvdlgv5"]

for cluster in not_found:
    if cluster.cluster_id.strip() in failed_pipelines:
        print(f"cluster in failed pipelines: {cluster.cluster_id.strip()}")

cluster in failed pipelines: 20oj6d80880k817hfh8dpp91s3c5is53
cluster in failed pipelines: 20ol00uiid1qe72enspo7rn3esvdlgv5


In [18]:
delete_events = [
    "26415702-e3fc-425e-b8fb-240eabd8a28b",
    "6b02bb42-9c2a-4916-94c9-d79d872ca585",
    "4b897ce1-3049-4183-a942-93346ac67c0e",
    "368f7362-e60a-4e01-a946-afd336c0a0e2",
    "7b005c31-969c-4001-bd03-5e9e5c90251f",
    "d241fbec-e0b3-4ddb-be72-80523bc97fad",
    "4749b6fa-2852-40e8-94e9-c50297c7141d",
    "206f8052-1adc-4230-a444-aa820a48f04d",
    "52a29a50-996a-47d6-8aff-ab8a62353fbe",
    "88b87be4-9d62-4977-a741-52f3a526f90b",
    "a617bc7b-4074-4135-ace8-0b3ff9115e9a",
    "ae3a135d-9669-48bd-8a62-fdf9d3ae5b22",
    "e6f86e0d-a691-4f2b-b4ee-d7c433584eb9",
    "e872a421-d81e-4b27-88f8-78f5d59bb068",
    "ee86bce0-0c9a-441f-a46b-4f0caac43398",
    "406091c2-bfe2-45a5-8bf8-856fa7910185",
    "bfe10363-ed98-42a8-ae4a-95159b9a2f6d",
    "79ef42ef-d166-4d87-8e6e-20d1728997e3",
    "f0650e97-46ce-4fd5-aa3e-fcf995e4dd85",
    "f2cbe11a-c1f1-4a59-94e9-b0cf42803639",
    "519ad4ae-2810-45fd-89f7-a10e986624e6",
    "76f65dc9-1894-4c9a-9484-91facb13469f",
    "581eec55-1dc4-403e-9a17-6162884163ad",
    "12630160-7ab7-4219-87b3-1118050aca93",
    "1c94f4b8-f73b-441f-a760-4f1268592466",
    "9a903ec4-3bd5-4738-9b55-20db5750397c",
    "ac8f28dc-4605-4baa-b886-1bfda1615d45",
    "ee9b67a7-9df9-4d58-920c-00a9d43620d2",
    "338c24ee-b518-4537-8d28-8326cb2bc730",
    "00b4ce69-f7f0-414b-9ed5-f85dc03c5bbf",
    "02c52928-1598-4141-b149-2953778c4e71",
    "e1f530de-6e51-4883-af6b-5578a2ead936",
    "8c3cbc24-83db-4081-b606-2b8df91f5f77",
    "ae02908c-4fe3-40a8-86ec-e2cd99710161",
    "d77e1fc1-b038-4ffe-9aaa-9a8014f4ec06",
    "954b4809-4fe5-4e1f-922b-0fa68f580122",
    "93dc8526-3760-415a-83d7-9fd51a720716",
    "1bc6c61e-a6f9-463e-a853-8191712b1e75",
    "72c97bf9-4fbd-4e17-9ea4-7df798fc5e8c",
    "477b7c28-8f97-4f83-a325-d433d2ee7b81",
    "7310c119-5b6c-4e45-982e-5ca8182c43d2",
    "1a763790-f612-466d-8379-19909a0e81c9",
    "7287eefb-ca0e-43e7-86f1-0ec0040221e7",
    "c3f1e111-71f1-4bc4-8792-8081a76ecaf2",
    "a41e6d56-3bb6-431c-8e66-d7b94ffeb7a0",
    "91431cb0-c331-4eec-9ec5-ebe0bf7ee0ea",
    "7c758ab3-ab0b-4437-bb3b-331a81b4d7e8",
    "6f6c4ced-3e8b-417a-8236-08fea14f01d9",
    "0b413f17-3a7a-47f1-b3d6-1b56e4cf5460",
    "712e67b6-28a3-4780-bba8-988106e8fa64",
    "b9f9e23c-51fb-4a3d-8619-7807afcbe073",
    "1bcbe504-b542-49eb-a3d4-6cc7df4da485",
    "6cd70a68-9b08-4500-82d1-b8e577c32c44",
    "691de290-28f3-4594-8d6a-6149dad53be4",
    "64aed4ae-487d-4623-9b95-30389f2170da",
    "09bc3a30-1515-433f-abad-8486b2f1f987",
    "430d5bdc-afbd-4abf-861f-89e497f9737d",
    "a3fe0323-4b99-4321-8120-da15419c8426",
    "af346c54-e46b-422a-9047-33e7d3b958a6",
    "e3a7537f-43bf-4c65-b7d9-d266a7cd605b",
    "200c1580-3c8b-443d-b336-fe2beca4efb1",
    "6d14bef7-b7c6-4e2a-b516-7075b2af1510",
    "8eb5e547-71d6-41a4-b070-d44cc8907827",
   ]

for cluster in not_found:
    if cluster.cluster_uuid.strip() in delete_events:
        print(f"Found : {cluster.cluster_uuid.strip()}")
else:
    print("No clusters found")

No clusters found
